Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

=====> /Users/rob/.julia/dev/StatisticalRethinking/scripts/04



Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)


make: `/Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/weights' is up to date.
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD        Naive SE        MCSE      ESS
alpha 154.5917285 0.27504113 0.00434878215 0.00359978864 1000
 beta   0.9060023 0.04329413 0.00068454035 0.00071964774 1000
sigma   5.1084553 0.19599336 0.00309892716 0.00307402424 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%        97.5%   
alpha 154.0619250 154.4060000 154.5930000 154.7760000 155.12202500
 beta   0.8191547   0.8759410   0.9060555   0.9352902   0.99022762
sigma   4.7396318   4.9739125   5.0997250   5.2396400   5.51042100



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.72 0.861507 5.05019; 154.405 0.944393 5.33055; … ; 154.313 0.905522 5.34465; 154.75 0.921736 4.91127]

Union{Missing, Float64}[153.707 0.917005 5.57356; 155.455 0.88529 4.62627; … ; 155.041 1.00098 4.95638; 155.071 0.88522 4.99162]

Union{Missing, Float64}[154.389 0.906678 4.94403; 154.641 0.885074 5.31245; … ; 154.744 0.869774 4.75489; 154.464 0.850825 4.7479]

Union{Missing, Float64}[154.455 0.941725 5.09855; 154.601 0.89429 4.86187; … ; 154.861 0.860396 5.16265; 154.353 0.971952 4.89095]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
          Mean        SD        Naive SE        MCSE      ESS
alpha 154.5917285 0.27504113 0.00434878215 0.00359978864 1000
 beta   0.9060023 0.04329413 0.00068454035 0.00071964774 1000
sigma   5.1084553 0.19599336 0.00309892716 0.00307402424 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%        97.5%   
alpha 154.0619250 154.4060000 154.5930000 154.7760000 155.12202500
 beta   0.8191547   0.8759410   0.9060555   0.9352902   0.99022762
sigma   4.7396318   4.9739125   5.0997250   5.2396400   5.51042100



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*